In [1]:
import pandas as pd
import numpy as np
import psycopg2
import math
from datetime import timedelta

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',
                database = 'inicudb')

cur  = con.cursor()

In [2]:
#uhidPatient = "RNEH.0000012581"
uhidPatient = "RSHI.0000012984"
#caseType = 'Death'
caseType = 'Discharge'
seperator = '_'

In [3]:
data = pd.read_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv')

In [4]:
data['series'] = data.hour_series

In [5]:
data.columns

Index(['Unnamed: 0', 'actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA',
       'gender', 'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength.1',
       'birthheadcircumference.1', 'inout_patient_status.1',
       'gestationweekbylmp.1', 'gestationdaysbylmp.1', 'baby_type.1',
       'central_temp.1', 'apgar_onemin.1', 'apgar_fivemin.1', 'apgar_tenmin.1',
       'motherage.1', 'conception_type.1', 'mode_of_delivery.1',
       'steroidname.1', 'numberofdose.1', 'los', 'day_1', 'hour_series',
       'hour', 'day_x', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp', 'day_y', 'time_divide', 'series'],
      dtype='object')

In [6]:
data.urine


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
133767    35.0
133768    35.0
133769    35.0
133770    35.0
133771    35.0
Name: urine, Length: 133772, dtype: float64

In [7]:
data.time_divide.unique()

array([0.0000000e+00, 4.4805680e+05, 3.5973958e+04, 7.2166090e+03,
       1.4415046e+04, 2.8800173e+04, 2.2699035e+04, 2.4799230e+03,
       1.0826918e+04, 3.5934450e+03, 7.1781200e+03, 1.4415863e+04,
       1.4398689e+04, 2.8788535e+04, 1.4415568e+04, 2.5195263e+04,
       3.5776170e+03, 1.4408112e+04, 2.1600443e+04, 2.1606547e+04,
       7.2312100e+03, 7.1840860e+03, 1.0790560e+04, 7.2124970e+03,
       1.1260018e+04, 1.3949751e+04, 1.7992093e+04, 1.0761424e+04,
       7.2313360e+03, 1.0816918e+04, 1.4351256e+04, 1.3020641e+04,
       1.2198283e+04, 7.1884450e+03, 1.1641587e+04, 1.7197120e+04,
       1.4347548e+04, 1.0823273e+04, 3.5639332e+05, 1.4401211e+04,
       7.2144940e+03, 7.1967300e+03, 6.9356650e+03, 2.6109400e+02,
       7.2315050e+03, 7.1985410e+03, 7.2363870e+03, 7.1479810e+03,
       7.1826630e+03, 1.4381459e+04, 1.4417531e+04, 7.1901230e+03,
       7.2463240e+03, 7.1732460e+03, 1.4654810e+04, 1.4126478e+04,
       1.0804378e+04, 3.6268460e+03, 7.1952280e+03, 1.0788039e

In [8]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [9]:
def urine_check(x):
    try:
        if x > 200:
            return x/10
        else:
            return x
    except:
        pass

In [10]:
def to_float(x):
    if x == '':
        return np.nan
    else:
        return float(x)

In [11]:
dg = data.copy()

In [12]:
dg['urine'] = dg['urine'].apply(to_float)

In [13]:
dg['urine'] = dg['urine'].apply(urine_check)

In [14]:
dg['urine_per_hour'] = dg['urine']/((dg['time_divide'])/(3600))

In [15]:
dt = dg.sort_values(by = ['uhid','hour_series'])

In [16]:
ds = pd.DataFrame()

In [17]:
for i in dt.uhid.unique():
    x = dt[dt['uhid']==i]
    x['urine_per_hour'].fillna(method='bfill',inplace=True)
    ds = ds.append(x,ignore_index=True)
       

In [18]:
cur1 = con.cursor()
cur1.execute("SELECT uhid,creationtime,modificationtime,ph,entrydate FROM apollo.nursing_bloodgas where uhid ='"+ uhidPatient+"'")
cols1 = list(map(lambda x: x[0],cur1.description))
ph = pd.DataFrame(cur1.fetchall(),columns=cols1)

In [19]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08
1,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47
2,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
3,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08
7,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24
8,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24
9,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24


In [20]:
def ph_func(x):
    try:
        p = float(x)
        if p>=6.80 and p<=7.8:
            return p
        else:
            return None
    except:
        pass

In [21]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08
1,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47
2,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
3,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08
7,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24
8,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24
9,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24


In [22]:
ph['new_ph'] = ph.ph.apply(ph_func)

In [23]:
ph.dropna(subset=['new_ph'],inplace=True)

In [24]:
ph_df = pd.DataFrame()
for i in ph.uhid.unique():
    x = ph[ph['uhid']==i]
    ph_df = ph_df.append(x,ignore_index=True)

In [25]:
ph

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
1,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
2,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
3,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
7,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
8,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
9,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31


In [26]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
1,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
2,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
3,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
7,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
8,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
9,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31


In [27]:
#ph_df = pd.DataFrame(columns = ph.columns)


In [28]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
1,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
2,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
3,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19
7,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
8,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31
9,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31


In [29]:
ph_df['hour'] = ph_df.entrydate.apply(split_hour)
ph_df.drop_duplicates(subset=['uhid','hour'],keep='last',inplace=True)

In [30]:
ds

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide,series,urine_per_hour
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:33:00.000,0.128555
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:34:00.000,0.128555
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:35:00.000,0.128555
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:36:00.000,0.128555
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:37:00.000,0.128555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133767,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,7196.508,2018-11-22 14:00:00.000,17.508492
133768,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:01:00.000,inf
133769,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:02:00.000,inf
133770,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:03:00.000,inf


In [31]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph,hour
4,RSHI.0000012984,2018-09-14 22:31:15,2018-10-06 19:39:46.186753,7.39,2018-09-14 22:31:15,7.39,2018-09-14 22
5,RSHI.0000012984,2018-09-11 12:32:47,2018-10-06 19:39:46.186753,7.32,2018-09-11 12:32:47,7.32,2018-09-11 12
6,RSHI.0000012984,2018-09-09 21:00:08,2018-10-06 19:39:46.186753,7.19,2018-09-09 21:00:08,7.19,2018-09-09 21
12,RSHI.0000012984,2018-12-31 04:37:24,2018-12-31 04:37:24.000000,7.31,2018-12-31 04:37:24,7.31,2018-12-31 04


In [32]:
type(ds.uhid[0])

str

In [33]:
ds['uhid'] = ds['uhid'].astype(str)


In [34]:
ss = pd.merge(ds,ph_df,on=['uhid','hour'],how='left')
ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')

In [35]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:33:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:34:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:35:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:36:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:37:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133767,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,7196.508,2018-11-22 14:00:00.000,17.508492,NaT,NaT,NaN,NaT,NaN
133768,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:01:00.000,inf,NaT,NaT,NaN,NaT,NaN
133769,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:02:00.000,inf,NaT,NaT,NaN,NaT,NaN
133770,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:03:00.000,inf,NaT,NaT,NaN,NaT,NaN


In [36]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:33:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:34:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:35:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:36:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,NaN,0.000,2018-08-21 16:37:00.000,0.128555,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133767,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,7196.508,2018-11-22 14:00:00.000,17.508492,NaT,NaT,NaN,NaT,NaN
133768,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:01:00.000,inf,NaT,NaT,NaN,NaT,NaN
133769,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:02:00.000,inf,NaT,NaT,NaN,NaT,NaN
133770,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:03:00.000,inf,NaT,NaT,NaN,NaT,NaN


In [37]:
len(ss.uhid.unique())

1

In [38]:
cur3 = con.cursor()
cur3.execute("SELECT t1.uhid,t1.entrydate,t1.rbs,t1.skintemp,t1.centraltemp  FROM apollo.nursing_vitalparameters AS t1 where t1.uhid = '"+uhidPatient+"'")
cols3 = list(map(lambda x: x[0], cur3.description))
nv = pd.DataFrame(cur3.fetchall(),columns=cols3)

In [39]:
nv.drop_duplicates(subset=['uhid','entrydate'],keep='first',inplace=True)

In [40]:
nv['hour'] = nv.entrydate.apply(split_hour)

In [41]:
nv

,uhid,entrydate,rbs,skintemp,centraltemp,hour
0,RSHI.0000012984,2018-08-26 18:00:17.495,NaN,None,36.6,2018-08-26 18
1,RSHI.0000012984,2018-08-28 03:00:52.902,NaN,None,NaN,2018-08-28 03
2,RSHI.0000012984,2018-08-28 04:00:58.734,NaN,None,NaN,2018-08-28 04
3,RSHI.0000012984,2018-08-28 05:00:42.045,NaN,None,NaN,2018-08-28 05
4,RSHI.0000012984,2018-08-28 01:00:24.777,NaN,None,NaN,2018-08-28 01
...,...,...,...,...,...,...
3370,RSHI.0000012984,2018-11-12 01:00:04.120,NaN,None,36.5,2018-11-12 01
3371,RSHI.0000012984,2019-01-13 02:00:22.195,NaN,None,36.6,2019-01-13 02
3372,RSHI.0000012984,2018-12-07 22:31:58.693,NaN,None,37.4,2018-12-07 22
3373,RSHI.0000012984,2018-12-07 23:07:20.860,NaN,None,36.6,2018-12-07 23


In [42]:
ds

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide,series,urine_per_hour
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:33:00.000,0.128555
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:34:00.000,0.128555
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:35:00.000,0.128555
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:36:00.000,0.128555
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2018-08-21 16:37:00.000,0.128555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133767,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,7196.508,2018-11-22 14:00:00.000,17.508492
133768,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:01:00.000,inf
133769,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:02:00.000,inf
133770,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,29.5,35.0,NaN,NaN,2018-11-22 14:00:37.691000+08:00,2018-11-22,0.000,2018-11-22 14:03:00.000,inf


In [43]:
nv.hour.iloc[0]

'2018-08-26 18'

In [44]:
ss.uhid = ss.uhid.astype(str)

In [45]:
ss.hour.iloc[0]

'2018-08-21 16'

In [46]:
s1 = pd.merge(ss,nv,on = ['uhid','hour'], how='left')

In [47]:
len(s1.uhid.unique())

1

In [48]:
s1.drop_duplicates(subset=['uhid','hour_series'],keep='first',inplace=True)

In [49]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,urine_per_hour,creationtime,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,0.128555,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,0.128555,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,0.128555,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,0.128555,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,0.128555,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140847,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,17.508492,NaT,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6
140848,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6
140849,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6
140850,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6


In [50]:
def temp_a(x,y):
    if x>0:
        return x
    else:
        return y

In [51]:
s1['temp'] = s1.apply(lambda x: temp_a(x['centraltemp'], x['skintemp']), axis=1)

In [52]:
cur2 = con.cursor()
cur2.execute("SELECT t1.uhid,t1.visitdate,t1.visittime,t1.currentdateweight, t1.currentdateheight  FROM apollo.baby_visit AS t1 where t1.uhid = '"+uhidPatient+"'")
cols2 = list(map(lambda x: x[0], cur2.description))
bv = pd.DataFrame(cur2.fetchall(),columns=cols2)

In [53]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,RSHI.0000012984,2018-10-14,06:42:00,1002.0,NaN
1,RSHI.0000012984,2018-10-15,06:43:00,1007.0,36.5
2,RSHI.0000012984,2018-10-13,07:01:00,987.0,NaN
3,RSHI.0000012984,2018-10-16,06:46:00,1040.0,NaN
4,RSHI.0000012984,2018-10-17,07:26:00,1050.0,NaN
...,...,...,...,...,...
137,RSHI.0000012984,2018-10-05,01:21:02.458964,898.0,NaN
138,RSHI.0000012984,2018-10-06,07:42:00,903.0,NaN
139,RSHI.0000012984,2018-10-10,05:00:00,954.0,NaN
140,RSHI.0000012984,2018-10-11,06:55:00,973.0,NaN


In [54]:
bv.sort_values('visitdate',inplace=True)

In [55]:
def stamp_1(x,y):
    return str(x) +" "+ str(y).split(":")[0]

In [56]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
101,RSHI.0000012984,2018-08-21,14:11:19.261000,691.0,NaN
111,RSHI.0000012984,2018-08-22,01:29:02.928203,720.0,NaN
112,RSHI.0000012984,2018-08-28,13:04:00,683.0,NaN
113,RSHI.0000012984,2018-08-29,05:00:00,664.0,NaN
114,RSHI.0000012984,2018-08-30,07:13:00,675.0,NaN
...,...,...,...,...,...
106,RSHI.0000012984,2019-01-28,06:18:00,3590.0,NaN
107,RSHI.0000012984,2019-01-29,06:18:00,3510.0,NaN
108,RSHI.0000012984,2019-01-30,06:48:00,3600.0,NaN
109,RSHI.0000012984,2019-01-31,07:01:00,3715.0,NaN


In [57]:
bv['hour'] = bv.apply(lambda x: stamp_1(x['visitdate'], x['visittime']), axis=1)

In [58]:
def to_str(x):
    return (str(x))

In [59]:
bv['day'] = bv['visitdate'].apply(to_str)

In [60]:
bv.drop_duplicates(subset=['uhid','visitdate'],keep='last',inplace=True)

In [61]:
s1['day'] = s1['hour_series'].apply(split_date_1)

In [62]:
s1.day.unique()

array(['2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24',
       '2018-08-25', '2018-08-26', '2018-08-27', '2018-08-28',
       '2018-08-29', '2018-08-30', '2018-08-31', '2018-09-01',
       '2018-09-02', '2018-09-03', '2018-09-04', '2018-09-05',
       '2018-09-06', '2018-09-07', '2018-09-08', '2018-09-09',
       '2018-09-10', '2018-09-11', '2018-09-12', '2018-09-13',
       '2018-09-14', '2018-09-15', '2018-09-16', '2018-09-17',
       '2018-09-18', '2018-09-19', '2018-09-20', '2018-09-21',
       '2018-09-22', '2018-09-23', '2018-09-24', '2018-09-25',
       '2018-09-26', '2018-09-27', '2018-09-28', '2018-09-29',
       '2018-09-30', '2018-10-01', '2018-10-02', '2018-10-03',
       '2018-10-04', '2018-10-05', '2018-10-06', '2018-10-07',
       '2018-10-08', '2018-10-09', '2018-10-10', '2018-10-11',
       '2018-10-12', '2018-10-13', '2018-10-14', '2018-10-15',
       '2018-10-16', '2018-10-17', '2018-10-18', '2018-10-19',
       '2018-10-20', '2018-10-21', '2018-10-22', '2018-

In [63]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp,temp,day
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2018-08-21
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2018-08-21
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2018-08-21
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2018-08-21
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2018-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140847,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6,36.6,2018-11-22
140848,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6,36.6,2018-11-22
140849,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6,36.6,2018-11-22
140850,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaT,NaN,NaT,NaN,2018-11-22 14:00:18.242,NaN,None,36.6,36.6,2018-11-22


In [64]:
s2 = pd.merge(s1,bv,on = ['uhid','day'], how='left')

In [65]:
len(s2.uhid.unique())

1

In [66]:
#Meds



In [67]:
cur7= con.cursor()
cur7.execute("SELECT t1.uhid,t1.startdate,t1.medicineorderdate,t1.medicinename,t1.medicationtype,t2.typevalue FROM apollo.baby_prescription AS t1 LEFT JOIN apollo.ref_medtype AS t2 ON t1.medicationtype = t2.typeid WHERE (t1.medicationtype = 'TYPE0001' OR t1.medicationtype = 'TYPE0009' OR t1.medicationtype = 'TYPE0004') and (t1.uhid ='"+uhidPatient+"')")
cols7 = list(map(lambda x: x[0], cur7.description))
meds = pd.DataFrame(cur7.fetchall(),columns=cols7)

In [68]:
meds

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue
0,RSHI.0000012984,2018-08-31 18:41:38.406000+08:00,2018-08-31 18:41:38.406000+08:00,Amikacin,TYPE0001,Antibiotics
1,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Ciprofloxacin,TYPE0001,Antibiotics
2,RSHI.0000012984,2018-09-11 16:47:25.581000+08:00,2018-09-11 16:47:25.581000+08:00,Vancomycin,TYPE0001,Antibiotics
3,RSHI.0000012984,2018-08-31 20:46:07.266000+08:00,2018-08-31 20:46:07.266000+08:00,Colisitin,TYPE0001,Antibiotics
4,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Meropenem,TYPE0001,Antibiotics
5,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Amikacin,TYPE0001,Antibiotics
6,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,Antibiotics
7,RSHI.0000012984,2018-11-30 05:40:43.979000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,Antibiotics
8,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Piperacilin Tazobactam,TYPE0001,Antibiotics
9,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Amikacin,TYPE0001,Antibiotics


In [69]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])


In [70]:
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)


In [71]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,hour
0,RSHI.0000012984,2018-08-31 18:41:38.406000+08:00,2018-08-31 18:41:38.406000+08:00,Amikacin,TYPE0001,1,2018-08-31 18
1,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Ciprofloxacin,TYPE0001,1,2018-08-23 11
2,RSHI.0000012984,2018-09-11 16:47:25.581000+08:00,2018-09-11 16:47:25.581000+08:00,Vancomycin,TYPE0001,1,2018-09-11 16
3,RSHI.0000012984,2018-08-31 20:46:07.266000+08:00,2018-08-31 20:46:07.266000+08:00,Colisitin,TYPE0001,1,2018-08-31 20
4,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Meropenem,TYPE0001,1,2018-08-23 11
5,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Amikacin,TYPE0001,1,2018-11-28 18
6,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-11-28 18
7,RSHI.0000012984,2018-11-30 05:40:43.979000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-11-30 05
8,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-12-08 13
9,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Amikacin,TYPE0001,1,2018-12-08 13


In [72]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)
s2.drop(['hour_x','hour_y'],axis=1,inplace=True)

In [73]:
s2['hour'] = s2.hour_series.apply(split_hour)

In [74]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,hour
0,RSHI.0000012984,2018-08-31 18:41:38.406000+08:00,2018-08-31 18:41:38.406000+08:00,Amikacin,TYPE0001,1,2018-08-31 18
1,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Ciprofloxacin,TYPE0001,1,2018-08-23 11
2,RSHI.0000012984,2018-09-11 16:47:25.581000+08:00,2018-09-11 16:47:25.581000+08:00,Vancomycin,TYPE0001,1,2018-09-11 16
3,RSHI.0000012984,2018-08-31 20:46:07.266000+08:00,2018-08-31 20:46:07.266000+08:00,Colisitin,TYPE0001,1,2018-08-31 20
4,RSHI.0000012984,2018-08-23 11:56:02.501000+08:00,2018-08-23 11:56:02.501000+08:00,Meropenem,TYPE0001,1,2018-08-23 11
5,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Amikacin,TYPE0001,1,2018-11-28 18
6,RSHI.0000012984,2018-11-28 18:10:07.020000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-11-28 18
7,RSHI.0000012984,2018-11-30 05:40:43.979000+08:00,2018-11-28 18:10:07.020000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-11-30 05
8,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Piperacilin Tazobactam,TYPE0001,1,2018-12-08 13
9,RSHI.0000012984,2018-12-08 13:46:22.731000+08:00,2018-12-08 13:46:22.731000+08:00,Amikacin,TYPE0001,1,2018-12-08 13


In [75]:
s2

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,rbs,skintemp,centraltemp,temp,day,visitdate,visittime,currentdateweight,currentdateheight,hour
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,NaN,NaN,2018-08-21,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,NaN,NaN,2018-08-21,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,NaN,NaN,2018-08-21,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,NaN,NaN,2018-08-21,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,NaN,NaN,2018-08-21,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133767,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,36.6,36.6,2018-11-22,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14
133768,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,36.6,36.6,2018-11-22,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14
133769,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,36.6,36.6,2018-11-22,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14
133770,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,NaN,None,36.6,36.6,2018-11-22,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14


In [76]:
s3 = pd.merge(s2,meds_dummies,on = ['uhid','hour'], how='left')

In [77]:
s3['day'] = s3['hour_series'].apply(split_date_1)

In [78]:
s3

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,visitdate,visittime,currentdateweight,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16,NaT,NaT,NaN,NaN,NaN
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16,NaT,NaT,NaN,NaN,NaN
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16,NaT,NaT,NaN,NaN,NaN
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16,NaT,NaT,NaN,NaN,NaN
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21,14:11:19.261000,691.0,NaN,2018-08-21 16,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133827,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14,NaT,NaT,NaN,NaN,NaN
133828,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14,NaT,NaT,NaN,NaN,NaN
133829,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14,NaT,NaT,NaN,NaN,NaN
133830,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22,06:20:00,1880.0,NaN,2018-11-22 14,NaT,NaT,NaN,NaN,NaN


In [79]:
#PN
cur8= con.cursor()
cur8.execute("SELECT t1.uhid,t1.entrydatetime,t1.totalparenteralvolume,t1.total_intake FROM apollo.babyfeed_detail AS t1 where t1.uhid = '"+uhidPatient+"'")
cols8 = list(map(lambda x: x[0], cur8.description))
pn = pd.DataFrame(cur8.fetchall(),columns=cols8)

In [80]:
pn['hour'] = pn.entrydatetime.apply(split_hour)
pn['total_intake'] = pn['total_intake'].astype(float)
pn['tpn-tfl'] = pn['totalparenteralvolume']/pn['total_intake']

In [81]:
s4 = pd.merge(s3,pn,on = ['uhid','hour'], how='left')

In [82]:
s4

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,hour,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,entrydatetime,totalparenteralvolume,total_intake,tpn-tfl
0,0,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21 16,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,1,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21 16,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,2,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21 16,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,3,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21 16,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,4,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-08-21 16,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134487,133767,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
134488,133768,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
134489,133769,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN
134490,133770,2018-08-21 12:07:00.000,RSHI.0000012984,2018-11-22 14:04:10.942,2018-08-21 16:33:00.000,Male,691.0,NaN,NaN,In Born,...,2018-11-22 14,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [83]:
len(s4.uhid.unique())

1

In [84]:
s4.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [85]:
q = pd.DataFrame()

In [86]:
for i in s4.uhid.unique():
    x = s4[s4['uhid']==i]
    x['currentdateheight'].fillna(method='ffill',inplace=True)
    x['currentdateweight'].fillna(method='ffill',inplace=True)
    q = q.append(x,ignore_index=True)

In [87]:
def abd(x):
    
    try:   
        
        if x>200:
            return x/10
        else:
            return x
    except:
        pass
    

In [88]:
q['abdomen_girth'] = pd.to_numeric(q['abdomen_girth'], errors='coerce')
q['abdomen_girth'] = q['abdomen_girth'].apply(abd)

In [89]:
dt = pd.DataFrame()
for i in q.uhid.unique():
    x = q[q['uhid']==i]
    #for j in y['day'].unique():
    #x = y[y['day']==j]
    x['abdomen_girth'].fillna(method='bfill',inplace=True)
    x['currentdateweight'].fillna(method='bfill',inplace=True)
    x['currentdateheight'].fillna(method='bfill',inplace=True)
    x['totalparenteralvolume'].fillna(method='ffill',inplace=True)
    x['total_intake'].fillna(method='ffill',inplace=True)
    x['tpn-tfl'].fillna(method='ffill',inplace=True)
    dt = dt.append(x,ignore_index=True)

In [90]:
dt.urine_per_hour.max()

inf

In [91]:
dt.urine_per_hour.unique()

array([1.28555130e-01,            inf, 9.00651521e-01, 1.99539701e+00,
       4.49530303e+00, 1.37499174e+00, 1.58597050e+00, 1.74198957e+01,
       3.32504596e+00, 1.40255382e+01, 4.01219261e+00, 3.49614865e+00,
       3.75034144e+00, 3.25129431e+00, 1.74811010e+00, 1.71460802e+00,
       6.03753839e+00, 1.49915548e+00, 6.66652994e-01, 8.33080825e-01,
       2.98705196e+00, 3.00664552e+00, 3.00262452e+00, 2.49566828e+00,
       2.55772238e+00, 4.12910596e+00, 1.60070315e+00, 4.01433862e+00,
       6.96966646e+00, 1.99687194e+00, 3.01018949e+00, 2.21187267e+00,
       2.36098802e+00, 7.51205581e+00, 3.09236189e+00, 2.51204853e+00,
       4.51645117e+00, 4.32401548e+00, 5.05059971e-02, 2.24981080e+00,
       3.99196395e+00, 4.00181749e+00, 4.67150590e+00, 1.37881376e+02,
       4.48039516e+00, 3.50070938e+00, 3.97988665e+00, 7.55458080e+00,
       4.51086178e+00, 2.50322307e+00, 1.49817607e+00, 4.00549476e+00,
       3.97442897e+00, 5.52051331e+00, 2.21087820e+00, 1.78388414e+00,
      

In [92]:
a = dt.replace([np.inf, -np.inf], np.nan)

In [93]:
def abd_2(x):
    if x < 0:
        return ((-1)*x)/10
    elif x > 200:
        return x/10
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_2)

def abd_3(x):
    if x < 15:
        return np.nan
    elif x > 50:
        return np.nan
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_3)

def upm(x):
    if x > 100:
        return np.nan
    else:
        return x
    
a['urine_per_hour'] = a['urine_per_hour'].apply(upm)

def temp_f_c(x):
    if x > 90:
        return (x - 32)*(5/9)
    else:
        return x
    
a['temp'] = a['temp'].apply(temp_f_c)

def weight_correct(x):
    if x<10:
        return x*100
    elif x>5000:
        return x/10
    else:
        return x
    
def weight_correct_2(x):
    if x<300:
        return x*10
    elif x>5000:
        return x/10
    else:
        return x
    
a.currentdate = a.currentdateweight.apply(weight_correct)
a.currentdate = a.currentdateweight.apply(weight_correct_2)

def height_correct(x):
    if x < 10:
        return np.nan
    else:
        return x
    
def rbs_correct(x):
    if x > 700:
        return np.nan
    else:
        return x
    
a['currentdateheight'] = a['currentdateheight'].apply(height_correct)

a['rbs'] = a['rbs'].apply(rbs_correct)



/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [94]:
len(a.uhid.unique())

1

In [95]:
data = a.copy()

In [96]:
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass

In [97]:
data['day'] = data['hour_series'].apply(split_date_1)

In [98]:
def to_date(x):
    return pd.to_datetime(x)

In [99]:
data['hour_series'] = data['hour_series'].apply(to_date)

In [100]:
len(data.uhid.unique())

1

In [101]:
data.to_csv(caseType+seperator+uhidPatient+seperator+"intermittent_checkpoint_2.csv")

In [ ]:
df = pd.DataFrame()
for i in data.uhid.unique():
    x = data[data['uhid']==i]
    x.rbs.fillna(method='ffill',inplace=True)
    x.currentdateweight.fillna(method='ffill',inplace=True)
    x.currentdateheight.fillna(method='ffill',inplace=True)
    n = math.ceil(len(x)/24)+1
    start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)
    for i in range(int(n)):
        
       
        y = x[(x['hour_series']>=start_date + timedelta(hours=24*i)) & (x['hour_series']<=start_date + timedelta(hours=24*(i+1)))]
        y['stool_day_total'] = y['stool_passed'].sum()
        df =df.append(y,ignore_index=True)


/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
df.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_3.csv')

In [ ]:
test = df.copy()

In [ ]:
dq = pd.DataFrame()

In [ ]:
for i in test.uhid.unique():
    x = test[test['uhid']==i]
    x.abdomen_girth.fillna(method='ffill',inplace=True)
    dq = dq.append(x, ignore_index=True)

In [ ]:
final = pd.DataFrame()

In [ ]:
for i in dq.uhid.unique():
    x = dq[dq['uhid']==i]
    x['abd_difference_y'] = x['abdomen_girth'] - x['abdomen_girth'].shift(periods=1)
    final = final.append(x,ignore_index=True)

In [ ]:
final.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_4.csv')

In [ ]:
cur1 = con.cursor()
cur1.execute("SELECT t1.uhid,t1.starttime,t1.pulserate, t1.ecg_resprate, t1.spo2, t1.heartrate, t2.dischargestatus,t1.mean_bp,t1.sys_bp,t1.dia_bp  FROM apollo.device_monitor_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"') OR (t2.dischargestatus = '"+caseType+"' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"');")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
ds.sort_values('starttime',inplace=True)

In [ ]:
ds

In [ ]:
test = ds.drop_duplicates(subset=['uhid','starttime'],keep='first')

In [ ]:
cur_vent = con.cursor()
cur_vent.execute("SELECT t1.uhid,t1.start_time,t1.creationtime,t1.peep, t1.pip,t1.map ,t1.tidalvol, t1.minvol,t1.ti,t1.fio2 FROM apollo.device_ventilator_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' OR t2.dischargestatus = '"+caseType+"') and t2.uhid = '"+uhidPatient+"';")
cols_vent = list(map(lambda x: x[0], cur_vent.description))
ventilator_cont = pd.DataFrame(cur_vent.fetchall(),columns=cols_vent)

test_vent = ventilator_cont.drop_duplicates(subset=['uhid','start_time'],keep='first')

def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

test['date'] = test.starttime.apply(split_date)
test_vent['date'] = test_vent.start_time.apply(split_date)

In [ ]:
cont_data = pd.merge(test,test_vent,on=['uhid','date'],how='left',copy=False)
test_cont = cont_data.drop_duplicates(subset=['uhid','starttime','heartrate'],keep='first')


In [ ]:
test_cont['hour_series'] = test_cont['date'].apply(split_hour)

In [ ]:
def to_str(x):
    return str(x)

In [ ]:
test_cont['ref_hour'] = test_cont['hour_series'].apply(to_str)
final['ref_hour'] = final['hour_series'].apply(to_str)

In [ ]:
def con_time(x):
    return str(x).split("+")[0].split(":")[0] + ":" +str(x).split("+")[0].split(":")[1]

In [ ]:
test_cont['cont_time'] = test_cont.starttime.apply(con_time)

In [ ]:
def con_time_2(x):
    return str(x).split(":")[0] +":"+str(x).split(":")[1]

In [ ]:
final['cont_time'] = final.ref_hour.apply(con_time_2)

In [ ]:
qw = pd.merge(final,test_cont, on=['uhid','cont_time'],how='left')

In [ ]:
#final[final['cont_time']=='2019-09-30 11:18']

In [ ]:
#qw.dropna(subset=['series'],inplace=True)

In [ ]:
qw.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_5.csv')